In [12]:
import numpy as np
import pandas as pd
from unidecode import unidecode

In [20]:
def read_to_list(path):
    out_list = []
    with open(path, 'r') as file:
        for line in file:
            out_list.append(line.rstrip())
    return out_list

def read_to_df(path):
    out_list = read_to_list(path)
    out_df = pd.DataFrame (out_list, columns=['strings'])
    return out_df

def clean_df(df_original):
    df = df_original.copy()
    df = df[df['strings'].str[0] != '#']
    df[['words','rest']] = df.strings.str.split('=',expand=True)
    df[['category', 'sources']] = df.rest.str.split(';fonts: ',expand=True)
    # df['category'] = df['category'].str.split('categories: ').apply(lambda x: x[1] if len(x) > 1 else None)
    df['category'] = df['category'].str.split('categories: ').apply(lambda x: None if x is None else (x[1] if len(x) > 1 else None))
    df.sources = df.sources.str[0:-1]
    df.drop(['strings','rest'],axis=1, inplace=True)
    df['words_std'] = df.words.apply(lambda x: unidecode(x).lower())
    return df


In [21]:
path = './Data/noms-fdic.txt'
noms_df = read_to_df(path)
noms_df = clean_df(noms_df)

path = './Data/adjectius-fdic.txt'
adj_df = read_to_df(path)
adj_df = clean_df(adj_df)

In [59]:
noms_df.words

191         17-A
192          17A
193          4x4
194           A3
195           A4
          ...   
87150         ús
87151    ústaixa
87152       úter
87153       úvea
87154      úvula
Name: words, Length: 86964, dtype: object

In [61]:
def find_words (df_words, main_letter, usable_letters, word_len, guess='', wordlist=[]):
    df = df_words.copy()
    df = df[df.words.str.contains(main_letter)]

    for letter in usable_letters:
        new_guess = guess + letter
        print(new_guess)
        if new_guess in list(df['words_std']):
            wordlist.append(new_guess)
        if len(new_guess) <= word_len:
            wordlist = find_words (df_words, main_letter, usable_letters, word_len, new_guess, wordlist)
    
    return wordlist 

def contains_letters (word, letters):
    for letter in word:
        if letter not in letters:
            return False
    return True

def find_words_fast (df_words, main_letter, usable_letters):
    wordlist = []
    df = df_words.copy()
    df = df[df.words.str.contains(main_letter)]
    for index, row in df.iterrows():
        if contains_letters (row['words_std'], usable_letters):
            wordlist.append(row['words'])
    return wordlist

In [62]:
main_letter = 'l'
usable_letters = ['l','u','b','s','a','t','o']
words_out_2 = find_words_fast (noms_df, main_letter, usable_letters)
# words_out = find_words_fast (adj_df, main_letter, usable_letters)

In [63]:
np.array(words_out_2)

array(['ababol', 'abatoll', 'absolta', 'ala', 'alba', 'albat', 'albó',
       'all', 'allassa', 'allau', 'alosa', 'alot', 'alou', 'alto',
       'aluata', 'aló', 'assalt', 'atall', 'atol', 'aula', 'aulos',
       'autoblast', 'aüllat', 'babol', 'babols', 'bala', 'balast',
       'balata', 'balau', 'balboa', 'ball', 'balla', 'balot', 'balou',
       'balsa', 'baló', 'basalt', 'bassal', 'bassol', 'batall', 'batalla',
       'batallola', 'batalló', 'batolla', 'baula', 'baül', 'blast',
       'blasó', 'blat', 'blau', 'blob', 'blàstula', 'bol', 'bola', 'boll',
       'bolla', 'bollat', 'bolló', 'bolo', 'bolsal', 'boló', 'botall',
       'botaló', 'boual', 'bulb', 'bull', 'busal', 'butla', 'butlla',
       'bàbol', 'búbal', 'lata', 'llast', 'llata', 'llau', 'llautó',
       'llaüt', 'lloa', 'lloba', 'llobat', 'llobató', 'llosa', 'llosat',
       'llossa', 'llot', 'loboa', 'lobus', 'lot', 'loto', 'lotus',
       'lòbul', 'löss', 'olla', 'ollal', 'ooblast', 'saballó', 'sal',
       'sala', 'sa

In [38]:
print(prova)

['lot', 'ull', 'ula', 'bol', 'sal', 'sol', 'all', 'ala', 'alo', 'tul', 'llau', 'lloa', 'llot', 'lata', 'loss', 'lot', 'loto', 'ull', 'ula', 'blau', 'blat', 'blob', 'bull', 'bulb', 'ball', 'bala', 'balo', 'baul', 'bol', 'boll', 'bola', 'bolo', 'sal', 'sala', 'salt', 'salo', 'sol', 'sola', 'solo', 'soul', 'all', 'alba', 'albo', 'ala', 'alto', 'alo', 'alou', 'alot', 'aula', 'atol', 'tul', 'tall', 'tala', 'talo', 'toll', 'tola', 'olla', 'obol']


In [23]:
def guess_number(target, guess=""):
    if guess == target:
        print("Target number found:", guess)
        return True
    elif len(guess) >= len(target):
        return False

    for digit in range(10):
        new_guess = guess + str(digit)
        if guess_number(target, new_guess):
            return True

    return False

# Example usage:
target_number = "1234"
guess_number(target_number)

Target number found: 1234


True